Data downloaded from https://www.kaggle.com/extralime/math-lectures/version/2

In [ ]:
import numpy as np
import pandas as pd
import nltk

Standardize

In [2]:
import random
import re
import string

In [3]:
# takes ~30 seconds to import
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

Vectorize

In [4]:
from sklearn.feature_extraction.text import CountVectorizer


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv_tfidf = TfidfVectorizer()

Reduce Dimensions

In [6]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF


Cluster

In [7]:
from sklearn.cluster import KMeans

Visualize

In [8]:
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/anaconda3/envs/metis/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Read in the data

In [2]:
df = pd.read_csv("raw_text.csv")

NameError: name 'pd' is not defined

In [3]:
df.head()

NameError: name 'df' is not defined

In [1]:
df.groupby('label').count()

NameError: name 'df' is not defined

In [11]:
df.iloc[0][0]

"The following content is\nprovided under a Creative Commons license. Your support will help\nMIT OpenCourseWare continue to offer high quality\neducational resources for free. To make a donation, or to\nview additional materials from hundreds of MIT courses,\nvisit MIT OpenCourseWare at ocw.mit.edu. PROFESSOR: So,\nProfessor Jerison is relaxing in sunny\nLondon, Ontario today and sent me in as\nhis substitute again. I'm glad to the here\nand see you all again. So our agenda today: he\nsaid that he'd already talked about power series\nand Taylor's formula, I guess on last week\nright, on Friday? So I'm going to go a\nlittle further with that and show you some examples,\nshow you some applications, and then I have this\ncourse evaluation survey that I'll hand out in the last\n10 minutes or so of the class. I also have this handout\nthat he made that says 18.01 end of term 2007. If you didn't pick this up\ncoming in, grab it going out. People tend not to pick it\nup when they walk in, I 

### investigate and remove stop words

In [ ]:
set(stopwords.words('english'))

# PIPELINE

FUNCTION 1: Standardize  
FUNCTION 2: Vectorize  
FUNCTION 3: Reduce Dimensions  
FUNCTION 4: Cluster/Visualize

## Function 1: Standardize

For Iteration 1, in addition to "\n", remove all stop words and punctuation (generic lists).

In [13]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
def standardize(df):
    # replace '\n' with whitespace
    repl = lambda x: re.sub('\n', ' ', x)
    
    # remove numbers
    alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
    # remove punctuation, convert to lowercase
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
    
    df['text'] = df.text.map(repl).map(alphanumeric).map(punc_lower)
    
    return df

In [15]:
df_01_s = standardize(df)

In [16]:
df_01_s.iloc[0][0]

'the following content is provided under a creative commons license  your support will help mit opencourseware continue to offer high quality educational resources for free  to make a donation  or to view additional materials from hundreds of mit courses  visit mit opencourseware at ocw mit edu  professor  so  professor jerison is relaxing in sunny london  ontario today and sent me in as his substitute again  i m glad to the here and see you all again  so our agenda today  he said that he d already talked about power series and taylor s formula  i guess on last week right  on friday  so i m going to go a little further with that and show you some examples  show you some applications  and then i have this course evaluation survey that i ll hand out in the last   minutes or so of the class  i also have this handout that he made that says     end of term    if you didn t pick this up coming in  grab it going out  people tend not to pick it up when they walk in  i see  so grab this when yo

## Function 2: Vectorize

For Iteration 1, use Tf-Idf vectorizer.
Also remove stop words within vectorizer.

In [21]:
def vectorize_tfidf(df):
    '''
    Creates a sparse matrix of counts with Tf-Idf Vectorizer
    '''
    # define series to go into vectorizer
    x = df['text']
    # define vectorizer
    cv_tfidf = TfidfVectorizer(stop_words='english')
    # vectorize; convert to sparse matrix "sm"
    sm = cv_tfidf.fit_transform(x)
    # print the matrix
    print(pd.DataFrame(sm.toarray(), columns=cv_tfidf.get_feature_names()))
    # return the sparse matrix
    return sm

In [24]:
sm_01_v = vectorize_tfidf(df_01_s)

      aa  aah  aardvark  aaron   ab  aba  abandon  abandoned  abbreviate  \
0    0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
1    0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
2    0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
3    0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
4    0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
..   ...  ...       ...    ...  ...  ...      ...        ...         ...   
855  0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
856  0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
857  0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
858  0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.000000         0.0   
859  0.0  0.0       0.0    0.0  0.0  0.0      0.0   0.025504         0.0   

     abbreviated  ...   zr   zs   zt  zuiden   zx  zxy   zy   zz  zzz  zzzzzz  
0      

In [25]:
sm_01_v.shape

(860, 21909)

## Function 3: Reduce Dimensions

In [37]:
def reduce_dim(sm):
    # define Truncated SVD
    lsa = TruncatedSVD(20)
    # do LSA on sparse matrix "sm"
    rd = lsa.fit_transform(sm)
    # print explained variance ratio
    print("LSA Explained Variance Ratio: ",lsa.explained_variance_ratio_)
    # return matrix of reduced dimensions
    return rd

In [38]:
rd_01_v = reduce_dim(sm_01_v)

LSA Explained Variance Ratio:  [0.01144721 0.03803046 0.02663736 0.02372008 0.01907087 0.01793495
 0.01647002 0.01413434 0.01352123 0.01215464 0.01175551 0.01107554
 0.01038059 0.01011165 0.00962199 0.00939259 0.00871361 0.00843379
 0.00804542 0.00790111]


In [39]:
rd_01_v

array([[ 0.31383981, -0.07075516, -0.0070351 , ..., -0.06274067,
        -0.0182043 ,  0.06356786],
       [ 0.0863835 , -0.05528352,  0.04099555, ..., -0.01808304,
         0.03799406,  0.0064178 ],
       [ 0.26250266, -0.10747971, -0.12898366, ..., -0.04046962,
         0.06653842,  0.04040354],
       ...,
       [ 0.43514768,  0.05732553, -0.03100395, ..., -0.02476448,
         0.02069879, -0.01700341],
       [ 0.22121884, -0.08790136, -0.15002771, ..., -0.03429434,
         0.28272234,  0.12349469],
       [ 0.49246248, -0.15187721, -0.06727716, ...,  0.01659677,
        -0.03202912,  0.1312628 ]])

### Look at the topics

In [29]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [31]:
x = df['text']
cv_tfidf = TfidfVectorizer(stop_words='english')
sm = cv_tfidf.fit_transform(x)
lsa = TruncatedSVD(5)
rd = lsa.fit_transform(sm)
display_topics(lsa, cv_tfidf.get_feature_names(), 5)


Topic  0
going, just, let, minus, right

Topic  1
matrix, vector, row, column, minus

Topic  2
probability, event, conditional, random, probabilities

Topic  3
laplace, transform, derivative, minus, integral

Topic  4
gt, vector, just, vectors, plane


In [33]:
x = df['text']
cv_tfidf = TfidfVectorizer(stop_words='english')
sm = cv_tfidf.fit_transform(x)
lsa = TruncatedSVD(20)
rd = lsa.fit_transform(sm)
display_topics(lsa, cv_tfidf.get_feature_names(), 10)


Topic  0
going, just, let, minus, right, vector, matrix, times, equal, plus

Topic  1
matrix, vector, row, column, minus, vectors, times, transpose, determinant, columns

Topic  2
probability, event, conditional, random, probabilities, events, equal, sample, minus, outcomes

Topic  3
laplace, transform, derivative, minus, integral, equation, squared, function, differential, plus

Topic  4
gt, vector, just, vectors, plane, gonna, okay, dot, line, length

Topic  5
gt, laplace, row, transform, minus, node, matrix, determinant, lambda, okay

Topic  6
vector, node, laplace, transform, probability, equal, tree, vectors, times, address

Topic  7
node, address, lambda, equation, linked, variable, okay, subtree, derivative, random

Topic  8
solution, equation, gt, minus, plus, differential, equal, derivative, audience, times

Topic  9
audience, gt, professor, vector, minus, lambda, log, squared, beta, random

Topic  10
gt, ok, integral, probability, event, transpose, delta, plane, space, zero


## Next Steps (for Iteration 02):
Create list of irrelevant words to remove from raw data.  
- going, just, let, minus, professor, audience, plus, okay, print, mit, respect, gonna, just, 

Create Compound Terms. (from nltk.tokenize import MWETokenizer # multi-word expression)  
- Laplace Transform
- Partial Derivative
- (variable) squared

Stem.  
- Probability vs. Probabilities
- Matrix vs. Matrices
- event vs. events
- edge vs. edges
- transform vs. transformation

In [50]:
# import these modules 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

ps = PorterStemmer() 

# choose some words to be stemmed 
words = ["matrix", "matrices", "transform", "transformation", "probability", "probabilities"] 

for w in words: 
	print(w, " : ", ps.stem(w)) 


matrix  :  matrix
matrices  :  matric
transform  :  transform
transformation  :  transform
probability  :  probabl
probabilities  :  probabl


In [52]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
words = ["matrix", "matrices", "transform", "transformation", "probability", "probabilities"] 

for w in words: 
	print(w, " : ", stemmer.stem(w)) 

matrix  :  matrix
matrices  :  mat
transform  :  transform
transformation  :  transform
probability  :  prob
probabilities  :  prob


In [53]:
from nltk.stem.snowball import SnowballStemmer
sb = SnowballStemmer("english")
words = ["matrix", "matrices", "transform", "transformation", "probability", "probabilities"] 

for w in words: 
	print(w, " : ", sb.stem(w)) 

matrix  :  matrix
matrices  :  matric
transform  :  transform
transformation  :  transform
probability  :  probabl
probabilities  :  probabl


In [57]:
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
wn = WordNetLemmatizer()

words = ["matrix", "matrices", "transform", "transformation", "probability", "probabilities"] 

for w in words: 
	print(w, " : ", wn.lemmatize(w)) 

matrix  :  matrix
matrices  :  matrix
transform  :  transform
transformation  :  transformation
probability  :  probability
probabilities  :  probability


In [ ]:
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
wn = WordNetLemmatizer()

words = ["matrix", "matrices", "transform", "transformation", "probability", "probabilities"] 

for w in words: 
	print(w, " : ", wn.lemmatize(w)) 